In [2]:
import sys
from pathlib import Path

src_path = Path("..")
sys.path.append(str(src_path.resolve()))

device = "cuda"
speaker_path = "/home/alexbarrow/proj/video-dubbing-dls-project/data/speaker_sample.wav"

# from IPython.display import Audio
from config import config

In [3]:
conf = config.load_config("../config/config.yaml")

### Pipeline